In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
import os
from tpot import TPOTRegressor
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import *
from sklearn.svm import *


/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
dfe_tmp = pd.read_csv('../raw_data/data_final_indé_big.csv')
dfe_tmp.head(2)

,steam_appid,name,top_5_tags,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,...,english,price,has_a_website,nb_review,owner_estimated,rating,popularity,score_descriptif,average_playtime,median_playtime
0,1002,Rag Doll Kung Fu,fighting multiplayer 1980s realistic real_time...,0.016212,0.016212,0.016212,0.016212,0.401611,0.016212,0.016212,...,1,5.99,1,57,22451.0,29.800000,45.100000,9,0,0
1,2710,Act of War: Direct Action,action strategy rts 1980s relaxing,0.016444,0.016444,0.016444,0.016444,0.016444,0.016444,0.016444,...,1,1.99,0,213,29159.0,1522.087755,1924.966327,4,0,0


In [8]:
dfe_tmp['price'][dfe_tmp['price'] == 0].shape

(1515,)

In [9]:
dfe_tmp['price'][dfe_tmp['price'] <= 1].shape

(4135,)

In [10]:
dfe_tmp['price'][dfe_tmp['price'] <= 2].shape

(6483,)

In [11]:
dfe_tmp['price'][dfe_tmp['price'] <= 3].shape

(8780,)

_____

In [3]:
dfe_tmp.columns

Index(['steam_appid', 'name', 'top_5_tags', 'topic_0', 'topic_1', 'topic_2',
       'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8',
       'topic_9', 'topic_10', 'topic_11', 'topic_12', 'topic_13', 'topic_14',
       'topic_15', 'topic_16', 'topic_17', 'topic_18', 'topic_19', 'english',
       'price', 'has_a_website', 'nb_review', 'owner_estimated', 'rating',
       'popularity', 'score_descriptif', 'average_playtime',
       'median_playtime'],
      dtype='object')

In [4]:
X = dfe_tmp.drop(columns=['steam_appid', 'top_5_tags', 'owner_estimated','name', 'rating', 'popularity', 'nb_review','average_playtime', 'median_playtime'])

In [5]:
X.columns

Index(['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5',
       'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10', 'topic_11',
       'topic_12', 'topic_13', 'topic_14', 'topic_15', 'topic_16', 'topic_17',
       'topic_18', 'topic_19', 'english', 'price', 'has_a_website',
       'score_descriptif'],
      dtype='object')

In [6]:
X.shape

(21547, 24)

In [7]:
y = dfe_tmp['owner_estimated']

In [8]:
y.shape

(21547,)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state=0)

In [10]:
num_transformer = make_pipeline(SimpleImputer(), StandardScaler())
num_col = make_column_selector(dtype_include=['float64','int64'])

cat_transformer = OneHotEncoder()
cat_col = make_column_selector(dtype_include=['object','bool'])

preproc_basic = make_column_transformer(
    (num_transformer, num_col),
    (cat_transformer, cat_col))

In [11]:
pipe = make_pipeline(preproc_basic, Ridge())

In [12]:
pipe.fit(X_train,y_train)

# Make predictions
pipe.predict(X_test)

# Score model
pipe.score(X_test,y_test)


0.1292074167661127

# TPOT AUTO ML

In [14]:
# instanciate TPOTClassifier
tpot = TPOTRegressor(generations=4, population_size=50, verbosity=2, scoring='r2', n_jobs=-1, cv=2)
# process autoML with TPOT
tpot.fit(X_train, y_train)

# print score
print(tpot.score(X_test, y_test))

Optimization Progress:   0%|          | 0/250 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.20301447587596172

Generation 2 - Current best internal CV score: 0.20301447587596172

Generation 3 - Current best internal CV score: 0.2060721435640006

Generation 4 - Current best internal CV score: 0.20608806925564804

Best pipeline: RandomForestRegressor(PolynomialFeatures(input_matrix, degree=2, include_bias=False, interaction_only=False), bootstrap=True, max_features=0.6000000000000001, min_samples_leaf=18, min_samples_split=4, n_estimators=100)
0.20404347960702407


# RandomForestRegressor

In [15]:
X_train.columns

Index(['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5',
       'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10', 'topic_11',
       'topic_12', 'topic_13', 'topic_14', 'topic_15', 'topic_16', 'topic_17',
       'topic_18', 'topic_19', 'english', 'price', 'has_a_website',
       'score_descriptif'],
      dtype='object')

In [16]:
forest = RandomForestRegressor(
                               bootstrap=True, 
                               max_features=0.4, 
                               min_samples_leaf=14, 
                               n_estimators=100,
                               min_samples_split=14)
model = forest

model.fit(X_train, y_train)

print(model.score(X_test,y_test))

0.20257635320348588
